In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row

In [2]:
from szp_funcs import get_fio, print_df, load_groups, gku_inn, path_to_docs, to_double, path_to_data, szp, create_full_res, create_res

In [3]:
months = ['jan', 'feb', 'mar']

In [44]:
grp_res = pd.read_excel(path_to_docs + 'school_type_grp.xlsx')

In [45]:
grouped_inns = grp_res.groupby('grp')

In [46]:
groups = list(grouped_inns.groups.keys())

In [47]:
groups

['ГПЦЦ', 'ДОУ', 'Колледжи', 'Школы']

In [48]:
list(grouped_inns['inn'].get_group(groups[1]))

[7731108147, 7713283643]

In [49]:
res = create_res(months)

In [50]:
res['szp'] = res.apply(lambda row: szp(row, months), axis=1)

In [51]:
res

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp
0,7720258623,001-005-586 64,140968.88,1.0,Заведующий хозяйством,124968.88,1.0,Заведующий хозяйством,NaN,NaN,NaN,132968.880000
1,7736614860,001-010-312 25,164214.30,1.0,Учитель,146547.14,1.0,Учитель,160599.30,1.0,Учитель,157120.246667
2,7723169690,001-011-996 67,161441.63,1.0,Учитель,167905.34,1.0,Учитель,166752.15,1.0,Учитель,165366.373333
3,7727123537,001-017-453 65,90135.84,1.0,Документовед,87135.84,1.0,Секретарь руководителя,86627.92,1.0,Секретарь руководителя,87966.533333
4,7733088746,001-017-484 72,88112.10,1.0,Учитель,NaN,NaN,NaN,86787.10,1.0,Учитель,87449.600000
...,...,...,...,...,...,...,...,...,...,...,...,...
153458,7734408766,215-089-180 55,NaN,NaN,NaN,NaN,NaN,NaN,25039.34,1.0,Специалист,25039.340000
153459,7736614860,215-093-175 39,NaN,NaN,NaN,NaN,NaN,NaN,38178.00,1.0,Уборщик территории,38178.000000
153460,7743020747,215-094-406 39,NaN,NaN,NaN,NaN,NaN,NaN,10015.73,1.0,Рабочий по комплексному обслуживанию и ремонту...,10015.730000
153461,7736050780,215-120-882 18,NaN,NaN,NaN,NaN,NaN,NaN,13916.40,1.0,Рабочий по комплексному обслуживанию и ремонту...,13916.400000


In [52]:
full_res = create_full_res(['mar'])

In [53]:
jobs = ['Учитель-логопед', 'Учитель-дефектолог', 'Сурдопедагог', 'Педагог-психолог ', 'Тьютор', 'Ассистент']

In [54]:
res_by_jobs = res[res.job_mar.isin(jobs)][['job_mar', 'szp', 'inn']].groupby('job_mar')

In [83]:
ans_szp = []
ans_stv_sum = []
ans_ppl_sum = []
ans_sums = []

In [56]:
full_res[full_res.job_mar == jobs[3]]

,inn,snils,sum_mar,stv_mar,job_mar
75,7715992532,001-102-475 52,42719.99,1.0,Педагог-психолог
129,7735062750,001-141-396 70,50380.00,1.0,Педагог-психолог
225,7736228173,001-170-048 64,118857.04,1.0,Педагог-психолог
322,7728900803,001-181-440 77,59627.20,1.0,Педагог-психолог
520,7726032196,001-199-304 06,85263.18,1.0,Педагог-психолог
...,...,...,...,...,...
166348,7734686844,212-494-290 48,96153.18,1.0,Педагог-психолог
166378,7713780268,212-904-986 58,69519.95,1.0,Педагог-психолог
166390,7733820977,213-022-377 95,107493.06,1.0,Педагог-психолог
166460,7705480398,213-716-972 60,48772.73,1.0,Педагог-психолог


In [84]:
ans_szp.append(res[(res.job_mar.isin(jobs)) & (res.inn.isin(list(grp_res['inn'])))][['job_mar', 'szp']].groupby('job_mar').mean())
ans_stv_sum.append(full_res[full_res.job_mar.isin(jobs) & (full_res.inn.isin(list(grp_res['inn'])))][['job_mar', 'stv_mar']].groupby('job_mar').sum())
ans_ppl_sum.append(full_res[full_res.job_mar.isin(jobs) & (full_res.inn.isin(list(grp_res['inn'])))][['job_mar', 'snils']].groupby('job_mar').count())

In [85]:
for group in groups:
    inns = list(grouped_inns['inn'].get_group(group))
    ans_szp.append(res[(res.inn.isin(inns)) & (res.job_mar.isin(jobs))][['job_mar', 'szp']].groupby('job_mar').mean())
    ans_stv_sum.append(full_res[(full_res.inn.isin(inns)) & (full_res.job_mar.isin(jobs))][['job_mar', 'stv_mar']].groupby('job_mar').sum())
    ans_ppl_sum.append(full_res[(full_res.inn.isin(inns)) & (full_res.job_mar.isin(jobs))][['job_mar', 'snils']].groupby('job_mar').count())

In [59]:
ans_szp[0]

,szp
job_mar,
Ассистент,69007.968851
Педагог-психолог,91894.418072
Сурдопедагог,110606.528333
Тьютор,88130.805595
Учитель-дефектолог,95330.496812
Учитель-логопед,89038.713251


In [60]:
ans_stv_sum[2]

,stv_mar
job_mar,
Педагог-психолог,1.0
Тьютор,1.0
Учитель-логопед,1.0


In [61]:
ans_ppl_sum[4]

,snils
job_mar,
Ассистент,168
Педагог-психолог,2685
Тьютор,1614
Учитель-дефектолог,1068
Учитель-логопед,3179


In [86]:
full_ans = pd.DataFrame()

In [87]:
for i in range(5):
    if i == 0:
        full_ans = pd.concat([full_ans, ans_szp[0]]).rename(columns={'szp':'szp_all'})
        full_ans = pd.merge(full_ans, ans_stv_sum[0], how='left', on='job_mar').rename(columns={'stv_mar':'stv_all'})
        full_ans = pd.merge(full_ans, ans_ppl_sum[0], how='left', on='job_mar').rename(columns={'snils':'ppl_all'})
        continue
    full_ans = pd.merge(full_ans, ans_stv_sum[i], how='left', on='job_mar').rename(columns={'stv_mar':'stv_' + str(i)})
    full_ans = pd.merge(full_ans, ans_ppl_sum[i], how='left', on='job_mar').rename(columns={'snils':'ppl_' + str(i)})
    full_ans = pd.merge(full_ans, ans_szp[i], how='left', on='job_mar').rename(columns={'szp':'szp_' + str(i)})

In [88]:
full_ans

,szp_all,stv_all,ppl_all,stv_1,ppl_1,szp_1,stv_2,ppl_2,szp_2,stv_3,ppl_3,szp_3,stv_4,ppl_4,szp_4
job_mar,,,,,,,,,,,,,,,
Ассистент,69007.968851,208.460000,256,NaN,NaN,NaN,NaN,NaN,NaN,47.0,88,84789.443889,161.460000,168.0,66022.284384
Педагог-психолог,91702.748098,3121.975000,3080,229.50,232.0,97890.792974,1.0,1.0,90641.00,162.0,162,104400.580351,2729.475000,2685.0,90418.199924
Сурдопедагог,110606.528333,3.000000,3,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3,110606.528333,NaN,NaN,NaN
Тьютор,88089.969081,1750.466667,1801,NaN,NaN,NaN,1.0,1.0,82379.08,184.2,186,114883.085971,1565.266667,1614.0,84656.421197
Учитель-дефектолог,94911.064138,1238.353000,1199,99.50,98.0,97902.481564,NaN,NaN,NaN,33.0,33,102851.880278,1105.853000,1068.0,94378.611002
Учитель-логопед,88932.901731,3529.596000,3406,177.75,175.0,95003.846655,1.0,1.0,NaN,51.0,51,93172.650631,3299.846000,3179.0,88518.757758


In [65]:
res[res.snils == '020-808-065 12']

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp
24011,7713283643,020-808-065 12,131299.51,1.0,Учитель-логопед,131523.19,1.0,Учитель-логопед,NaN,NaN,NaN,131411.35


In [89]:
print_df(full_ans, 'Учереждения с ОВЗ')

In [67]:
pg = full_res.copy()

In [68]:
pg = pg[(pg.inn.isin(list(grp_res['inn']))) & (pg.job_mar.isin(jobs))]

In [69]:
pg

,inn,snils,sum_mar,stv_mar,job_mar
43,7724731696,001-070-338 65,90870.00,1.0,Учитель-логопед
53,7725144330,001-076-189 94,100696.97,1.0,Учитель-логопед
75,7715992532,001-102-475 52,42719.99,1.0,Педагог-психолог
129,7735062750,001-141-396 70,50380.00,1.0,Педагог-психолог
133,7735002230,001-142-160 56,124080.00,1.5,Учитель-дефектолог
...,...,...,...,...,...
166384,7723398058,212-971-553 60,77900.00,1.0,Учитель-логопед
166390,7733820977,213-022-377 95,107493.06,1.0,Педагог-психолог
166460,7705480398,213-716-972 60,48772.73,1.0,Педагог-психолог
166482,7733857776,214-009-561 17,17118.20,1.0,Педагог-психолог


In [4]:
res_all = create_full_res(months)

In [5]:
ped, isp, aup = load_groups('input')

In [6]:
isp

['Преподаватель',
 'Воспитатель',
 'Инструктор по физической культуре',
 'Концертмейстер',
 'Мастер производственного обучения',
 'Музыкальный руководитель',
 'Педагог дополнительного образования',
 'Педагог-организатор',
 'Педагог-психолог ',
 'Преподаватель ',
 'Социальный педагог',
 'Тьютор',
 'Учитель',
 'Учитель-дефектолог',
 'Учитель-логопед',
 'Помощник воспитателя',
 'Младший воспитатель']

7733672292 - шк 1387

7734687492 - Курчатовская

7726287412 - 657

7743024068 - 1583

In [7]:
inn_of_int = 7743024068

In [8]:
all = sum(list(res_all[res_all.inn == inn_of_int][['sum_' + month for month in months]].sum()))

In [9]:
res_all[res_all.inn == inn_of_int].count()

inn        319
snils      319
sum_jan    315
stv_jan    315
job_jan    315
sum_feb    304
stv_feb    301
job_feb    301
sum_mar    302
stv_mar    301
job_mar    301
dtype: int64

In [10]:
all

86762681.28

In [14]:
isp_sum = sum(list(res_all[(res_all.inn == inn_of_int) & ((res_all.job_mar.isin(isp)))][['sum_' + month for month in months]].sum()))

In [15]:
isp_sum

64816536.06

In [39]:
print_df(res_all[(res_all.inn == inn_of_int)][['sum_' + month for month in months] + ['snils', 'inn'] + ['job_' + month for month in months]], 'ppl kurch mar')

In [16]:
isp_sum / all

0.7470554748166953

In [17]:
isp_sum / 87499000

0.7407688780443206

In [90]:
print_df(pd.merge(pg, res[['inn', 'snils', 'szp']], how='left', on=['inn', 'snils']), 'Люди для учреждения с ОВЗ')

In [71]:
pd.merge(pg, res[['inn', 'snils', 'szp']], how='left', on=['inn', 'snils'])

,inn,snils,sum_mar,stv_mar,job_mar,szp
0,7724731696,001-070-338 65,90870.00,1.0,Учитель-логопед,87860.000000
1,7725144330,001-076-189 94,100696.97,1.0,Учитель-логопед,99696.970000
2,7715992532,001-102-475 52,42719.99,1.0,Педагог-психолог,88236.465000
3,7735062750,001-141-396 70,50380.00,1.0,Педагог-психолог,63172.916667
4,7735002230,001-142-160 56,124080.00,1.5,Учитель-дефектолог,120879.583333
...,...,...,...,...,...,...
9740,7723398058,212-971-553 60,77900.00,1.0,Учитель-логопед,76566.666667
9741,7733820977,213-022-377 95,107493.06,1.0,Педагог-психолог,102780.096667
9742,7705480398,213-716-972 60,48772.73,1.0,Педагог-психолог,54500.000000
9743,7733857776,214-009-561 17,17118.20,1.0,Педагог-психолог,162565.850000
